## Stats701-001 Homework 3, Winter 2018
### Ben Denis Shaffer
#### bdshaff@umich.edu

### 1 More Fun with Tuples

Function `my_sum` is function that takes any number of arguments and returns their sum. Here arguments are assumed to be integers or real numbers.

In [8]:
def my_sum(*args):
    return(sum(args))

Function `reverse_tuple` takes a tuple as an argument and returns a tuple of the same length, with each element in reverse order.

In [43]:
def reverse_tuple(tp):
    j = len(tp) - 1
    l = []
    while j >= 0:
        l.append(tp[j])
        j = j - 1
    return(tuple(l))

Function `rotate_tuple` takes a tuple and an integer `n` as two arguemnts and returns a tuple rotated to the right. If `n` is negative left rotation takes place.

In [119]:
def rotate_tuple(tp,n):
    if type(tp) != tuple:
        raise TypeError("First argument must be of type tuple.")
    
    try:
        nr = n % len(tp)
        t = tp[-nr:] + tp[:-nr]
        return(t)
    except TypeError:
        if type(n) != int:
            print("An int was expected as the second argument. Trying to cast to int.")
            nr = int(n) % len(tp)
            t = tp[-nr:] + tp[:-nr]
            return(t)

### 2 More Fun with Vectors

Function `is_valid_sparse_vector` checks if the argument is a valid sparse vector, with keys or type int and values of type float only.

In [151]:
def is_valid_sparse_vector(v):
    l = len(v)
    if type(v) != dict:
        return(False)
    if sum([type(v[i]) == float for i in v.keys()]) != l:
        return(False)
    if sum([type(i) == int for i in v.keys()]) != l:
        return(False)

    return(True)

Function `sparse_inner_product` imlements the dot product of two valid sparse vectors.

In [175]:
def sparse_inner_product(v1,v2):
    res = 0
    if not is_valid_sparse_vector(v1) or not is_valid_sparse_vector(v2):
        raise TypeError("Both vectors must be valid sparce vectors.")
    else:
        ks = [k for k in v1.keys() if k in v2.keys()]
        res = sum([v1[ksi]*v2[ksi] for ksi in ks])
    
    return(res)

### 3 Counting Word Bigrams

In [1]:
#reading in the line with UTF-8 characters to strip from a line when cleaning
some_chars = open("badstring.txt", "r")
some_chars = some_chars.readline()

Function `clean_line` takes a single line of text as an argument, prefromes desired cleaning by splitting and stripping the line of text, and returns the clean line.

In [2]:
def clean_line(line):
    cl = ""
    for w in line.split():
        cl = cl + " " +  w.strip(some_chars).lower()
    return(cl.strip())

Function `count_bigrams_in_file` takes in a file name as an argument and retruns a dictionary of bigrams in the file. If the file is not found an error is raised. The file is assumed to be a text file.

In [3]:
def count_bigrams_in_file(file_name):
    res = {}
    try:
        #First count the total numbe of lines to use later in a while loop.
        num_lines = sum([1 for i in open(file_name,"r")])
        
        #initiate a dictionary with bigrams from the first two lines.
        cnt = num_lines - 2
        f = open(file_name,"r")

        f1 = f.readline()
        f2 = f.readline()
        
        #concatinating two lines into one to count cases with first word of second line.
        #and last word of the preceeding line.
        new_line = clean_line(f1) + " " + clean_line(f2)   #calling clean_line.
        new_line = new_line.strip()                        #stripping the while spaces.
        line_split = new_line.split()
        for i in range(len(line_split) - 1):
            tpl = (line_split[i],line_split[i+1])
            if tpl not in res.keys():
                res[tpl] = 1
            else:
                res[tpl] = res[tpl] + 1

        while cnt > 0:

            prev_line = new_line
            last_word = prev_line.split()[-1]              #saving the last word of the latest read line.
            f1 = f.readline()
            new_line = last_word + " " + clean_line(f1)    #concatinating the last word of precceding line to new line.
            new_line = new_line.strip()

            if new_line == prev_line:   #This is to avoid double counting due to blank lines, where the last_word would be blank and a line woudl be repeated.
                        next
            else:
                line_split = new_line.split()
                for i in range(len(line_split) - 1):
                    tpl = (line_split[i],line_split[i+1])
                    if tpl not in res.keys():
                        res[tpl] = 1
                    else:
                        res[tpl] = res[tpl] + 1

            cnt = cnt - 1
        
        f.close()
        
    except:
        print("File not found!!")
    
    return(res)

In [4]:
count_bigrams_in_file("testtxt.txt") #looks like it is functioning correctly

{('a', 'league'): 3,
 ('all', 'in'): 1,
 ('and', 'then'): 1,
 ('broo', 'cmno'): 1,
 ('death', 'rode'): 1,
 ('half', 'a'): 3,
 ('hundred', 'something'): 1,
 ('in', 'the'): 1,
 ('league', 'half'): 2,
 ('league', 'onward'): 1,
 ('more', 'and'): 1,
 ('of', 'death'): 1,
 ('onward', 'all'): 1,
 ('rode', 'the'): 1,
 ('six', 'hundred'): 1,
 ('something', 'more'): 1,
 ('the', 'six'): 1,
 ('the', 'valley'): 1,
 ('then', 'whatevet'): 1,
 ('valley', 'of'): 1,
 ('whatevet', 'broo'): 1}

Function `collocations` forms a list for each word in a text file containing all words that are adjacent to it in the file. IMPORTANT (This function depends on the `count_bigrams_in_file` function defined above!)

In [5]:
def collocations(file_name):
    clocres = {}
    res = count_bigrams_in_file(file_name) # first compute all bigrams

    #set first or second word of bigram counts if its not a key already.
    #and add the second to the list of colocated words at the same time.
    for k in res.keys():
        if k[0] not in clocres:
            clocres[k[0]] = set()
            clocres[k[0]].add(k[1])
        else:                         #if the word already has a key, add the second word to the list
            clocres[k[0]].add(k[1])

        # for cases where a word first appears as the second word of a bigram.
        if k[1] not in clocres:
            clocres[k[1]] = set()
            clocres[k[1]].add(k[0])
        else:
            clocres[k[0]].add(k[1])
    
    return(clocres)

In [6]:
collocations("testtxt.txt") #looks like it is functioning correctly too.

{'a': {'half', 'league'},
 'all': {'in'},
 'and': {'more', 'then'},
 'broo': {'cmno', 'whatevet'},
 'cmno': {'broo'},
 'death': {'of', 'rode'},
 'half': {'a'},
 'hundred': {'six', 'something'},
 'in': {'all', 'the'},
 'league': {'a', 'half', 'onward'},
 'more': {'and'},
 'of': {'death'},
 'onward': {'all'},
 'rode': {'death', 'the'},
 'six': {'hundred', 'the'},
 'something': {'hundred', 'more'},
 'the': {'six', 'valley'},
 'then': {'whatevet'},
 'valley': {'of'},
 'whatevet': {'broo'}}

In [8]:
import pickle

In [10]:
pickle.dump(count_bigrams_in_file("mobydick.txt"), open("mb.bigrams.pickle", "wb"))
pickle.dump(collocations("mobydick.txt"), open("mb.colloc.pickle", "wb"))

In [11]:
dd2 = pickle.load(open("mb.bigrams.pickle", "rb"))
dd1 = pickle.load(open("mb.colloc.pickle", "rb"))